# Определение региона, где добыча нефти принесет наибольшую прибыль. Анализ возможной прибыли и рисков. 

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных:
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей)

Условия:
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.


## Загрузка и подготовка данных

In [96]:
# импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
import numpy as np
import seaborn as sns

In [97]:
# прочитаем данные и создадим датафреймы
geo_data_0 = pd.read_csv('geo_data_0.csv')
geo_data_1 = pd.read_csv('geo_data_1.csv')
geo_data_2 = pd.read_csv('geo_data_2.csv')

In [98]:
# выведем на экран первые 5 строк таблиц для каждого из трех регионов
display(geo_data_0.head())
display(geo_data_1.head())
display(geo_data_2.head())


,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04


,id,f0,f1,f2,product
0,kBEdx,-15.00,-8.28,-0.01,3.18
1,62mP7,14.27,-3.48,1.00,26.95
2,vyE1P,6.26,-5.95,5.00,134.77
3,KcrkZ,-13.08,-11.51,5.00,137.95
4,AHL4O,12.70,-8.15,5.00,134.77


,id,f0,f1,f2,product
0,fwXo0,-1.15,0.96,-0.83,27.76
1,WJtFt,0.26,0.27,-2.53,56.07
2,ovLUW,0.19,0.29,-5.59,62.87
3,q6cA6,2.24,-0.55,0.93,114.57
4,WPMUX,-0.52,1.72,5.90,149.60


Изучим данные

In [99]:
print(geo_data_0.info())
print(geo_data_0.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
             f0        f1        f2   product
count 100000.00 100000.00 100000.00 100000.00
mean       0.50      0.25      2.50     92.50
std        0.87      0.50      3.25     44.29
min       -1.41     -0.85    -12.09      0.00
25%       -0.07     -0.20      0.29     56.50
50%        0.50      0.25      2.52     91.85
75%        1.07      0.70      4.72    128.56
max        2.36      1.34     16.00    185.36


In [100]:
print(geo_data_1.info())
print(geo_data_1.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
             f0        f1        f2   product
count 100000.00 100000.00 100000.00 100000.00
mean       1.14     -4.80      2.49     68.83
std        8.97      5.12      1.70     45.94
min      -31.61    -26.36     -0.02      0.00
25%       -6.30     -8.27      1.00     26.95
50%        1.15     -4.81      2.01     57.09
75%        8.62     -1.33      4.00    107.81
max       29.42     18.73      5.02    137.95


In [101]:
print(geo_data_2.info())
print(geo_data_2.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
             f0        f1        f2   product
count 100000.00 100000.00 100000.00 100000.00
mean       0.00     -0.00      2.50     95.00
std        1.73      1.73      3.47     44.75
min       -8.76     -7.08    -11.97      0.00
25%       -1.16     -1.17      0.13     59.45
50%        0.01     -0.01      2.48     94.93
75%        1.16      1.16      4.86    130.60
max        7.24      7.84     16.74    190.03


In [104]:
# проверим на дубликаты и уникальные значения
print(geo_data_0.duplicated().sum())
print(geo_data_1.duplicated().sum())
print(geo_data_2.duplicated().sum())
print(geo_data_0.nunique())
print(geo_data_1.nunique())
print(geo_data_2.nunique())

0
0
0
id          99990
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64
id          99996
f0         100000
f1         100000
f2         100000
product        12
dtype: int64
id          99996
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64


Все датасеты имеют одинаковую размерность. По 5 столбцов и по 100 000 строк. Столбцы: f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы), product — объём запасов в скважине (тыс. баррелей), столбец "id" - уникальный идентификатор местрождения. Столюец "id" не несет значимых признаков для обучения модели, удалим его. Типы данных: столбец "id" - это "object", остальные столбцы "float64". Пропусков в данных нет, дубликатов в нужных столбцах нет. Названия столбцов корректны. 

In [105]:
# создадим новые датафреймы, в них будут данные которые необходимы для обучения модели (удалим столбец "id")
df_0 = geo_data_0.drop('id', axis=1)
df_1 = geo_data_1.drop('id', axis=1)
df_2 = geo_data_2.drop('id', axis=1)
df_0.head()

,f0,f1,f2,product
0,0.71,-0.50,1.22,105.28
1,1.33,-0.34,4.37,73.04
2,1.02,0.15,1.42,85.27
3,-0.03,0.14,2.98,168.62
4,1.99,0.16,4.75,154.04


### Вывод. 
Данные изучены, подготовлены, переходим к обучению и проверке модели.

## Обучение и проверка модели

Разобьем данные для каждого региона на две выборки: обучающую и валидационную. Разобьем их в отношении 75:25

In [106]:
# Разделим датафрейм на features и target - целевой признак
target_0 = df_0['product']
features_0 = df_0.drop('product', axis=1)

# Разделим датафрейм на обучающую и валидационную выборку
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

In [107]:
# выведем размерности полученных переменных
var_list_0 = [features_0_train, features_0_valid, target_0_train, target_0_valid]
for var in var_list:
    print(var.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [108]:
# Разделим датафрейм на features и target - целевой признак
target_1 = df_1['product']
features_1 = df_1.drop('product', axis=1)

# Разделим датафрейм на обучающую и валидационную выборку
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

In [109]:
# выведем размерности полученных переменных
var_list_1 = [features_1_train, features_1_valid, target_1_train, target_1_valid]
for var in var_list:
    print(var.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [110]:
# Разделим датафрейм на features и target - целевой признак
target_2 = df_2['product']
features_2 = df_2.drop('product', axis=1)

# Разделим датафрейм на обучающую и валидационную выборку
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [111]:
# выведем размерности полученных переменных
var_list_2 = [features_2_train, features_2_valid, target_2_train, target_2_valid]
for var in var_list:
    print(var.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


Для обучения модели подходит только линейная регрессия, остальные недостаточно предсказуемые. Обучим модель линейной регрессии.Предсказания модели сохраним в переменной predictions. В переменной result сохраним показатель среднеквадратической ошибки RMSE для целевого признака валидационной выборки и предсказаний модели.

### Модель 0

In [122]:
model_0 = LinearRegression()
model_0.fit(features_0_train, target_0_train)
predictions_0 = pd.Series(model_0.predict(features_0_valid))
result_0 = mse(target_0_valid, predictions_0)**.5

print('RMSE', result_0)
print('Prediction_mean', predictions_0.mean())

RMSE 37.5794217150813
Prediction_mean 92.59256778438038


### Модель 1

In [120]:
model_1 = LinearRegression()
model_1.fit(features_1_train, target_1_train)
predictions_1 = pd.Series(model_1.predict(features_1_valid))
result_1 = mse(target_1_valid, predictions_1)**.5

print('RMSE', result_1)
print('Prediction_mean', predictions_1.mean())

RMSE 0.8930992867756155
Prediction_mean 68.72854689544602


### Модель 2

In [121]:
model_2 = LinearRegression()
model_2.fit(features_2_train, target_2_train)
predictions_2 = pd.Series(model_2.predict(features_2_valid))
result_2 = mse(target_2_valid, predictions_2)**.5

print('RMSE', result_2)
print('Prediction_mean', predictions_2.mean())

RMSE 40.02970873393434
Prediction_mean 94.96504596800489


Создадим таблицу из полученных результатов

In [116]:
predictions_list = [predictions_0, predictions_1, predictions_2]

In [124]:
df = {'value':['rmse', 'predictions_mean', 'product_mean', 'product_min', 'product_max'],
        'df_0':[result_0, predictions_0.mean(), df_0['product'].mean(), df_0['product'].min(),
                      df_0['product'].max()],
        'df_1': [result_1, predictions_1.mean(), df_1['product'].mean(), df_1['product'].min(),
                       df_1['product'].max()],
        'df_2': [result_2, predictions_2.mean(), df_2['product'].mean(), df_2['product'].min(),
                       df_2['product'].max()]}
        

    
results = pd.DataFrame(df)

In [125]:
results

,value,df_0,df_1,df_2
0,rmse,37.58,0.89,40.03
1,predictions_mean,92.59,68.73,94.97
2,product_mean,92.50,68.83,95.00
3,product_min,0.00,0.00,0.00
4,product_max,185.36,137.95,190.03


Проверим, как коррелируют между собой признаки

In [126]:
corr = df_0.corr()
corr.style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1,-0.440723,-0.00315334,0.143536
f1,-0.440723,1,0.00172443,-0.192356
f2,-0.00315334,0.00172443,1,0.483663
product,0.143536,-0.192356,0.483663,1


In [127]:
corr = df_1.corr()
corr.style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1,0.182287,-0.00177704,-0.0304905
f1,0.182287,1,-0.00259532,-0.0101549
f2,-0.00177704,-0.00259532,1,0.999397
product,-0.0304905,-0.0101549,0.999397,1


In [128]:
corr = df_2.corr()
corr.style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1,0.000528283,-0.000448133,-0.00198706
f1,0.000528283,1,0.000778661,-0.00101239
f2,-0.000448133,0.000778661,1,0.445871
product,-0.00198706,-0.00101239,0.445871,1


В случае df_1 видна почти прямая взаимосвязь между признаком f2  и целевым признаком product. Возможно именно из-за этого "Модель 1" сработала с лучшим качеством, значение rmse(квадратный корень из средней квадратичной ошибки) минимальное. В df_0 и df_2 взяимосвязь между f2  и product тоже есть. Значения в обих случаях близки.

### Вывод
Мы обучили и проверили 3 модели. Наилучшее качество показала "Модель 1".

## Подготовка к расчёту прибыли

Прочитаем еще раз техзадание и соберем исходные данные в одном месте.

In [129]:
# бюджет на разработку - 10 млрд. руб.
total_budget = 10000000000

# ожидаемая прибыль от единицы продукта - 450 тыс. руб.
profit_per_product_unit = 450000

# количество месторождений для разработки - 200
number_of_wells = 200

Подсчитаем минимальный объем сырья, необходимый для безубыточности

In [130]:
minimal_size = total_budget / (profit_per_product_unit * number_of_wells)
minimal_size

111.11111111111111

Такой подсчет показывает нам, где находится точка безубыточности. Ниже этого порога разрабатывать месторождения нерентабельно. 

Для дальнейших рассчетов сбросим индексы целевого признака в валидационной выборке для каждого региона и переведем предсказания в валидационной выборке в pd.Series

In [131]:
# Сбросим индексы целевого признака в валидационной выборке для каждого региона:
target_0_valid.reset_index(drop=True, inplace=True)
target_1_valid.reset_index(drop=True, inplace=True)
target_2_valid.reset_index(drop=True, inplace=True)

# Переведем предсказания в валидационной выборке в pd.Series:
predictions_0 = pd.Series(predictions_0)
predictions_1 = pd.Series(predictions_1)
predictions_2 = pd.Series(predictions_2)

## Расчёт прибыли и рисков 

Для подсчета операционной прибыли создадим функцию revenue, которая принимает на вход три параметра (target, predictions, count), производит сортировку одного из параметров (predictions_sorted) по убыванию, производит подбор значений второго параметра (selected) по соответствию индексов с индексами первого параметра в количестве величины третьего параметра. Возвращает функция значение формулы: разница произведения суммарных запасов по n точкам с доходностью от единицы объема и общих затрат на разработку. Т.е. потенциальный доход минус необходимые расходы, получаем операционную прибыль.

In [132]:
def revenue(target, predictions, count):
    predictions_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected = target[predictions_sorted.index][:count]
    return profit_per_product_unit * selected.sum() - total_budget

Создадим функцию "revenue_bootstrap". Используем метод "Bootstrap" и проанализируем 1000 выборок из валидационной переменной целевого признака в размере 500 объектов. В теле функции обратимся к написанной ранее функции revenue, а также вычислим риск убытка, равный процентному соотношению количества отрицательных значений к общему количеству значений показателя "выручка". Функция возвращает средний показатель прибыли, нижнюю и верхнюю границы 95%-го доверительного интервала, а также риск убытка.

In [133]:
state = np.random.RandomState(123)
def revenue_bootstrap(target, predictions, count):    
    values = []
    loss_counter = 0
    for i in range(1000):
        target_subsample = target.sample(n=count, replace=True, random_state=state)
        predictions_subsample = predictions[target_subsample.index] 
        revenue_result = revenue(target_subsample, predictions_subsample, number_of_wells)
        values.append(revenue_result)
        if revenue_result  < 0:
            loss_counter += 1
    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    mean = values.mean()
    return mean, lower, upper, (loss_counter/len(values)*100)

**geo_data_0**

In [141]:
mean_0, lower_0, upper_0, loss_risk_0 = revenue_bootstrap(target_0_valid, predictions_0, 500)
print('Средняя выручка:', mean_0)
print('2,5%-квантиль:', lower_0)
print('97,5%-квантиль:', upper_0)
print('Риск убытка', loss_risk_0, '%')

Средняя выручка: 406548059.6278489
2,5%-квантиль: -113539755.53735322
97,5%-квантиль: 935150640.4351168
Риск убытка 5.800000000000001 %


**geo_data_1**

In [142]:
mean_1, lower_1, upper_1, loss_risk_1 = revenue_bootstrap(target_1_valid, predictions_1, 500)
print("Средняя выручка:", mean_1)
print("2,5%-квантиль:", lower_1)
print("97,5%-квантиль:", upper_1)
print('Риск убытка', loss_risk_1, '%')

Средняя выручка: 524889125.11341953
2,5%-квантиль: 115897854.22751169
97,5%-квантиль: 942275507.4795445
Риск убытка 0.5 %


**geo_data_2**

In [143]:
mean_2, lower_2, upper_2, loss_risk_2 = revenue_bootstrap(target_2_valid, predictions_2, 500)
print("Средняя выручка:", mean_2)
print("2,5%-квантиль:", lower_2)
print("97,5%-квантиль:", upper_2)
print('Риск убытка', loss_risk_2, '%')

Средняя выручка: 421571288.5949606
2,5%-квантиль: -166935840.85818946
97,5%-квантиль: 970260163.0417635
Риск убытка 7.3 %


Соберем полученные данные в таблицу research_result

In [144]:
result = {'value':['revenue_mean', 'revenue_total', '0.025_quantile',
                        '0.975_quantile', 'loss_risk'],
        'geo_data_0':[mean_0, mean_0*number_of_wells, lower_0, upper_0, loss_risk_0],
        'geo_data_1': [mean_1, mean_1*number_of_wells, lower_1, upper_1, loss_risk_1],
        'geo_data_2': [mean_2, mean_2*number_of_wells, lower_2, upper_2, loss_risk_2]}
        

    
research_result = pd.DataFrame(result)

In [145]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [146]:
research_result

,value,geo_data_0,geo_data_1,geo_data_2
0,revenue_mean,406548059.63,524889125.11,421571288.59
1,revenue_total,81309611925.57,104977825022.68,84314257718.99
2,0.025_quantile,-113539755.54,115897854.23,-166935840.86
3,0.975_quantile,935150640.44,942275507.48,970260163.04
4,loss_risk,5.80,0.50,7.30


- Величина риска убытка должна быть не более 2.5%. Этому условию удовлетворяет только регион "geo_data_1" со знчением "0,5". Два других региона имеют риски 5,8% и 7.3% соответственно.
- среднее значение прибыли наибольшее в регионе "geo_data_1".

### **На основании проведенной работы по анализу возможной прибыли и рисков представленных 3-х месторождений, можно сделать вывод, что к разработке рекомендуется регион "geo_data_1".**